<a href="https://colab.research.google.com/github/Wenjia1215/Wenjia_ML_DDoS_experiments/blob/main/Wenjia's_6th_ML_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><strong>Wenjia's 6th ML experiment</strong></h1>


8/12/2022
</br>calculate the mutual information score of every feature, and then choose high scored features to train the model.


There is an error in the fifth experiment, the label is also put into X while calculating MI.

This time it's correct.

8/14/2022
<br/>update:
<br/>SVM, LR, KNN, every classifier is used to fit 3 models with 3 different feature sets.
<br/>
3*3 = 9

Goal: use Traffic data to predict which one is DDoS attack and which one is normal traffic.</br>
Outline:</br>

    1, Gathering Data

    2, Preparing Data

    3, Model Selection

    4, Training

    5, Evaluation

    6, Parameter Tuning

    7, Prediction

1, Gathering Data (Done by dataset lab)</br>
2, Preparing Data 

In [ ]:
!gdown --id 1V9Kn7QL6knFDB1TRSIPqm0Jne5beDHZP

In [ ]:
import pandas as pd

# see complete rows in pandas
# use pd.set_option('max_colwidth', <width>) for column width & pd.set_option('max_rows', <rows>) for number of rows.

pd.set_option('max_rows', 80)

In [ ]:
file_path = '/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'

In [ ]:
ddos_data = pd.read_csv(file_path)

In [ ]:

ddos_data.head()

Preprocess data</br>

In [ ]:
#data preprocessing
# NaN
# infinity

# Deleted
  # Flow ID
  # Source IP
  # Source Port
  # Destination IP
  # Timestamp
  # Protocol

**bold text**
Replacing missing values

In [ ]:
ddos_data.isnull().sum().sum()

In [ ]:
ddos_data = ddos_data.fillna(ddos_data.median())

In [ ]:
ddos_data.isnull().sum().sum()

  Replace Infinity values with "0"


In [ ]:
import numpy as np
ddos_data.isin([np.inf, -np.inf]).sum().sum()


In [ ]:
ddos_data.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
ddos_data.isin([np.inf, -np.inf]).sum().sum()

In [ ]:
ddos_data = ddos_data.replace(to_replace = "BENIGN", value = 0)
ddos_data = ddos_data.replace(to_replace = "DDoS", value = 1)

To make sure all data meaningful,</br>
Drop missing value again.

In [ ]:
#count the total amount of row
ddos_data.shape[0]

In [ ]:
# drop missings
ddos_data = ddos_data.dropna(axis=0)

In [ ]:
#count again
ddos_data.shape[0]
# the number is same, it proves that the data preprocess is efficient.

Preprocess IP address<br/>
To prevent having many features you have to encode IP addresses in your training data and treat new IP as unknown
<br/> Hot encoding


CIC-IDS2017

In [ ]:
# Pros:
# 1, The model can more easily identify IP's that belong to the same subnet.
# 2, The number of features remains small even for a large number of distinct IP addresses in your training data.
# Reference: https://www.quora.com/How-can-I-deal-with-IP-addresses-in-machine-learning-algorithms-in-traffic-analysis-and-anomaly-detection

In [ ]:
# from sklearn import preprocessing
# ip_encoder = preprocessing.LabelBinarizer()

In [ ]:
# ip_encoder.fit_transform(ddos_data.Flow_ID)

In [ ]:
ddos_data.head()

In [ ]:
list(ddos_data.columns)

In [ ]:
y = ddos_data[' Label']
# print(y)

Feature selection

# feature selection


In [ ]:
cols = []
# for row in ddos_data:
#         cols.append(row)
        
# print(cols)
# print(len(cols))
cols = list(ddos_data.columns)
print(cols,'\n',len(cols))

In [ ]:
# calculate the mutual information score of every feature


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)



# Utility functions from Tutorial
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores of All Features ")
    # plt.annotate(label, # this is the text
    #               (x,y), # these are the coordinates to position the label
    #               textcoords="offset points", # how to position the text
    #               xytext=(0,10), # distance from text to points (x,y)
    #               ha='center') # horizontal alignment can be left, right or center



In [ ]:
# To start, let's review the meaning of mutual information by looking at a few features from this dataset.
features = ['Fwd IAT Total']
sns.relplot(
    x="value", y=" Label", col="variable", data=ddos_data.melt(id_vars=" Label", value_vars=features), facet_kws=dict(sharex=False),
);

In [ ]:

features = [' Fwd Packet Length Max']
sns.relplot(
    x="value", y=" Label", col="variable", data=ddos_data.melt(id_vars=" Label", value_vars=features), facet_kws=dict(sharex=False),
);

In [ ]:
features = [' Active Std']
sns.relplot(
    x="value", y=" Label", col="variable", data=ddos_data.melt(id_vars=" Label", value_vars=features), facet_kws=dict(sharex=False),
);

1. The ddos dataset has seventy-eight features
2. Identify the features with the most potential.

3. Use the make_mi_scores function (introduced in the tutorial) to compute mutual information scores for the ddos features:

In [ ]:
#  = ddos_data
X_allFeatures = ddos_data.drop(columns = ' Label')

In [ ]:
mi_scores = make_mi_scores(X_allFeatures, y)

In [ ]:

# print all 78 features
print(mi_scores.head(78))


In [ ]:

f = open("MI_dictionary.txt","w")
f. write( str(mi_scores) )
f. close()

In [ ]:
plt.figure(dpi=90, figsize=(9, 14.5))
plot_mi_scores(mi_scores.head(78))


In [ ]:
# print(type(mi_scores)
for k,v in mi_scores.items():
  if v >= 0.5:
    print(k)

In [ ]:
# # using 16 features selected in paper "Proceedings 2020, 63, 51; doi:10.3390/proceedings2020063051"
# target_features_1 = [' Destination Port','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',' Fwd Packet Length Mean','Bwd Packet Length Max',
#                    ' Bwd Packet Length Mean',
#                    ' Fwd Header Length', ' Bwd Header Length',' Average Packet Size',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' Fwd Header Length',' Subflow Fwd Bytes',' Subflow Bwd Bytes',
#                    'Init_Win_bytes_forward']

In [ ]:
target_features_1 = []
for k,v in mi_scores.items():
  if v >= 0.5:
    target_features_1.append(k)
print(len(target_features_1))
print(target_features_1)

In [ ]:
# print(type(mi_scores)
target_features_2 = []
for k,v in mi_scores.items():
  if v >= 0.2:
    target_features_2.append(k)

print(target_features_2)

In [ ]:
# target_feature_2 = [
#     'Fwd IAT Total',                   
#  'Fwd IAT Max',
#  'Flow Duration',
#  'Flow IAT Max',
#  'Bwd IAT Max',
# 'Bwd IAT Total',
#  Subflow Fwd Bytes              0.660043
# Total Length of Fwd Packets     0.660043
#  Average Packet Size            0.561962
#  Destination Port               0.560211
#  Fwd Header Length              0.549934
#  Fwd Header Length.1            0.549934
#  Total Length of Bwd Packets    0.549766
#  Subflow Bwd Bytes              0.549766
#  Avg Bwd Segment Size           0.543892
#  Bwd Packet Length Mean         0.541197
# Bwd Packet Length Max           0.534044
# Init_Win_bytes_forward          0.511645
#  Fwd Packet Length Max          0.506397
#  Fwd Packet Length Mean         0.489626
#  Avg Fwd Segment Size           0.486742
#  Bwd Header Length              0.481922
#  Fwd IAT Mean                   0.448178
#  Active Min                     0.443257
#  Active Max                     0.438722
#  Init_Win_bytes_backward        0.420977
#  Total Fwd Packets              0.417196
# Subflow Fwd Packets             0.417196
#  act_data_pkt_fwd               0.406938
#  Fwd IAT Std                    0.404124
#  Packet Length Mean             0.394575
#  Packet Length Variance         0.368877
#  Packet Length Std              0.367918
#  Bwd Packet Length Std          0.350168
#  Fwd Packet Length Std          0.349950
#  Bwd Packets/s                  0.342176
#  Total Backward Packets         0.334701
#  Subflow Bwd Packets            0.334701
#  Bwd IAT Mean                   0.330619
#  Bwd Packet Length Min          0.315814
# Fwd Packets/s                   0.312244
#  Max Packet Length              0.302470
# Flow Bytes/s                    0.276330
#  Flow IAT Std                   0.250023
#  Flow IAT Mean                  0.242019
#  Bwd IAT Std                    0.239326
#  Flow Packets/s 
# ]

In [ ]:
# features MI greater than 0.5
X1 = ddos_data[target_features_1]
# features MI greater than 0.2
X2 = ddos_data[target_features_2]
# all features
X3 = X_allFeatures
X3.head()

In [ ]:
print(len(X1.columns), len(X1))
print(len(X2.columns), len(X2))
print(len(X3.columns), len(X3))

In [ ]:
print(X1.columns)

In [ ]:
X2.head()

4, Training


Devide data to two part, one for training, one for testing(predicting)

In [ ]:
# split the data into validation data & training data, for both feature and target
from sklearn.model_selection import train_test_split
# split the data using feature set X1
train_X1, validate_X1, train_y1, validate_y1 = train_test_split(X1, y, random_state = 0)

In [ ]:
# split the data using feature set X2
train_X2, validate_X2, train_y2, validate_y2 = train_test_split(X2, y, random_state = 0)

In [ ]:
# split the data using all features X3
train_X3, validate_X3, train_y3, validate_y3 = train_test_split(X3, y, random_state = 0)

In [ ]:
print('x1:', train_X1)
print('x2:', train_X2)
print('x3:', train_X3)

In [ ]:
# define model
from sklearn.tree import DecisionTreeRegressor
ddos_model = DecisionTreeRegressor()
# fit model
ddos_model.fit(train_X1, train_y1)

4.1 (Experimental) prediction

In [ ]:
# # get predicted Label on validation data
ddosLabel_prediction = ddos_model.predict(validate_X1)

In [ ]:
print(ddosLabel_prediction)
len(ddosLabel_prediction)

In [ ]:
# Evaluate: Determine how accurate the model's predictions are
# by using Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mean_absolute_error(validate_y1, ddosLabel_prediction)

Using K-Nearest Neighbors (KNN)


In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn_model_1 = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn_model_1.fit(train_X1, train_y1)

#Predict the response for test dataset
y_pred_knn_1 = knn_model_1.predict(validate_X1)

In [ ]:
mean_absolute_error(validate_y1, y_pred_knn_1)

KNN X2 model

In [ ]:


#Create KNN Classifier
knn_model_2 = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn_model_2.fit(train_X2, train_y2)

#Predict the response for test dataset
y_pred_knn_2 = knn_model_2.predict(validate_X2)

#calculating MAE
mean_absolute_error(validate_y2, y_pred_knn_2)

KNN X3 MODEL

In [ ]:


#Create KNN Classifier
knn_model_3 = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn_model_3.fit(train_X3, train_y3)

#Predict the response for test dataset
y_pred_knn_3 = knn_model_3.predict(validate_X3)

#calculating MAE
mean_absolute_error(validate_y3, y_pred_knn_3)

Using SVM
<br/>svm x1 model

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
svm_model_1 = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
svm_model_1.fit(train_X1, train_y1)

#Predict the response for test dataset
y_pred_svm_1 = svm_model_1.predict(validate_X1)

In [ ]:
mean_absolute_error(validate_y1, y_pred_svm_1)

Using SVM
<br/>svm X2 model


In [ ]:
#Create a svm Classifier
svm_model_2 = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
svm_model_2.fit(train_X2, train_y2)

#Predict the response for test dataset
y_pred_svm_2 = svm_model_2.predict(validate_X2)
mean_absolute_error(validate_y2, y_pred_svm_2)

Using SVM
<br/>svm X3 model


In [1]:
#Create a svm Classifier
svm_model_3 = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
svm_model_3.fit(train_X3, train_y3)

#Predict the response for test dataset
y_pred_svm_3 = svm_model_3.predict(validate_X3)
mean_absolute_error(validate_y3, y_pred_svm_3)

NameError: ignored

Using LogisticRegression
<BR/>X1 MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
LR_model_1 = LogisticRegression()
LR_model_1.fit(train_X1, train_y1)
LR_predictions_1 = LR_model_1.predict(validate_X1)
mean_absolute_error(validate_y1, LR_predictions_1)

x2 MODEL

In [ ]:
LR_model_2 = LogisticRegression()
LR_model_2.fit(train_X2, train_y2)
LR_predictions_2 = LR_model_2.predict(validate_X2)
mean_absolute_error(validate_y1, LR_predictions_2)

x3 MODEL

In [ ]:
LR_model_3 = LogisticRegression()
LR_model_3.fit(train_X3, train_y3)
LR_predictions_3 = LR_model_3.predict(validate_X3)
mean_absolute_error(validate_y3, LR_predictions_3)